This is the notebook that I used to generate my classification model on

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd 
import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 # MobileNetV2

In [3]:
# For reproducability
seed = 12222022
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
#define base_directory where Alzheimer_s Dataset is located
base_directory = 'Alzheimer_s Dataset'
base_directory = '/content/drive/MyDrive/' + base_directory #for google drive

In [ ]:
#code from guide to define data sets, modified for Drive
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0)
train_ds = image_generator.flow_from_directory(batch_size=16,
                                                 directory= base_directory+'/train',
                                                 shuffle=True,
                                                 target_size=(224,224), 
                                                 subset="training",
                                                 class_mode='categorical')
image_generator = ImageDataGenerator(rescale=1/255,validation_split=0.2) 
val_ds = image_generator.flow_from_directory(batch_size=16,
                                                 directory= base_directory+'/test',
                                                 shuffle=True,
                                                 target_size=(224,224),
                                                 class_mode='categorical')

#I prefer my own code
# batch_size = 32
# img_height = 224
# img_width = 224

# train_dir = "/content/drive/MyDrive/Alzheimer_s Dataset/train" 
# test_dir = "/content/drive/MyDrive/Alzheimer_s Dataset/test"

# train_ds = tf.keras.utils.image_dataset_from_directory(
#     train_dir,
#     validation_split=0.1,  
#     subset='training', 
#     shuffle=True,
#     seed = seed,
#     image_size=(img_height, img_width),
#     batch_size=batch_size,
#     color_mode="rgb",
# )

# val_ds = tf.keras.utils.image_dataset_from_directory(
#     test_dir,
#     validation_split=0.9,
#     subset="validation",
#     shuffle=True,
#     seed = seed,
#     image_size=(img_height, img_width),
#     batch_size=batch_size,
#     color_mode="rgb",
# )

# class_names = np.array(train_ds.class_names)
# num_classes = len(class_names)
# print(class_names)

In [ ]:
# #From one of my previous projects, code I found online did not inlude obvious buffer optimizations 
# data_augmentation = tf.keras.Sequential([
#   # tf.keras.layers.RandomFlip("horizontal"),
#   # tf.keras.layers.RandomRotation(0.02),
#   # tf.keras.layers.RandomContrast(factor=0.2),
#   # tf.keras.layers.RandomFlip(mode="horizontal"),
#   # tf.keras.layers.RandomRotation(factor=0.05),
#   #tf.keras.layers.RandomZoom(height_factor=(-0.1, -0)),
#   #tf.keras.layers.RandomContrast(factor=0.05),
# ])

# AUTOTUNE = tf.data.AUTOTUNE

# def aug(ds):
#   # Use data augmentation only on the training set.
#   ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
#   # Use buffered prefetching on all datasets.
#   return ds

# # train_ds = aug(train_ds)

# normalization_layer = tf.keras.layers.Rescaling(1./255)
# train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
# val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# for image_batch, labels_batch in train_ds:
#   print(image_batch.shape)
#   print(labels_batch.shape)
#   break

In [ ]:
# #Defining the model with summary of structure
# METRICS = [
#     keras.metrics.AUC(name='auc'),
#     "acc",
# ]

# base_model = MobileNetV2(input_shape=(224,224,3), weights='imagenet', include_top=False)

# base_model.trainable = False

# inputs = tf.keras.Input(shape=(img_height, img_width, 3))

# #x = base_model.layers[-1].output 
# #x = base_model.output
# x = Flatten()(base_model.output)

# #x = tf.keras.layers.GlobalAveragePooling2D()(x)

# #convolutional layer before final layer
# x = tf.keras.layers.Dense(32, activation = "relu")(x)

# #predictions = tf.keras.layers.Dense(4, activation= "softmax")(x)
# predictions = tf.keras.layers.Dense(num_classes)(x)
# model = Model(inputs = base_model.input, outputs = predictions)
# #model = tf.keras.Model(inputs, outputs = predictions)

# #model = base_model
# model.summary()

In [ ]:
# #Training the model
# base_learning_rate = 1e-2
# model.compile(
#   optimizer=tf.keras.optimizers.Adam(
#        learning_rate=base_learning_rate
#   ),
#   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#   metrics=METRICS,  
# )

# #another attempt to avoid overfitting
# earlycallback = tf.keras.callbacks.EarlyStopping(
#     monitor ="val_loss", 
#     #monitor = "val_acc",
#     min_delta = 0.0005,
#     mode ="auto", patience = 3,
#     verbose = True,
#     restore_best_weights = True
# )

# NUM_EPOCHS = 1000

# history = model.fit(train_ds, validation_data=val_ds, epochs=NUM_EPOCHS, callbacks=earlycallback)

In [6]:
base_model = MobileNetV2(input_shape=(224,224,3), weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False
x = Flatten()(base_model.output)

prediction = Dense(4, activation='softmax')(x)

modelvgg = Model(inputs=base_model.input, outputs=prediction)

modelvgg.compile(optimizer='adam',
loss=tf.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc'),'acc'])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=8,
                                            restore_best_weights=True)

modelvgg.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=callback)

Epoch 1/50
321/321 [==============================] - 2615s 8s/step - loss: 2.7130 - auc: 0.8281 - acc: 0.6407 - val_loss: 3.3432 - val_auc: 0.7897 - val_acc: 0.5825
Epoch 2/50
321/321 [==============================] - 18s 56ms/step - loss: 1.0141 - auc: 0.9392 - acc: 0.8266 - val_loss: 4.8442 - val_auc: 0.7562 - val_acc: 0.5747
Epoch 3/50
321/321 [==============================] - 18s 55ms/step - loss: 0.5803 - auc: 0.9682 - acc: 0.8875 - val_loss: 4.1594 - val_auc: 0.8100 - val_acc: 0.6458
Epoch 4/50
321/321 [==============================] - 17s 53ms/step - loss: 0.2703 - auc: 0.9870 - acc: 0.9320 - val_loss: 2.4739 - val_auc: 0.8537 - val_acc: 0.6904
Epoch 5/50
321/321 [==============================] - 18s 55ms/step - loss: 0.4821 - auc: 0.9738 - acc: 0.9098 - val_loss: 8.8059 - val_auc: 0.7226 - val_acc: 0.5504
Epoch 6/50
321/321 [==============================] - 17s 53ms/step - loss: 0.2717 - auc: 0.9868 - acc: 0.9473 - val_loss: 5.1924 - val_auc: 0.7736 - val_acc: 0.6005
Epoc

For saving the model

In [7]:
export_path = "models/output"
modelvgg.save(export_path+'AlzheimersMRI.h5')

For testing the model on single images

In [ ]:
from tensorflow.keras.preprocessing.image import load_img , img_to_array

from PIL import Image

In [ ]:
#picking from train to confirm classes notation
filename = base_directory+"/train/ModerateDemented/moderateDem18.jpg"
img = load_img(filename , color_mode="rgb",target_size = (224 , 224))
img = img_to_array(img)
img = img.reshape(1 , 224 ,224 ,3)
img = img.astype('float32')
img = img/255.0
result = model.predict(img)